In [ ]:
#Imrul
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os


model = tf.keras.models.load_model('/content/flower_model_final.keras')






/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:

# Print the summary
model.summary()
num_classes = model.output_shape[1]
print("Number of classes predicted by the model:", num_classes)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 286, 274, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 143, 137, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 141, 135, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 70, 67, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 68, 65, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 34, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 139264)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │      17,825,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 16)                  │           2,064 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,842,466 (136.73 MB)

 Trainable params: 17,921,232 (68.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,921,234 (68.36 MB)

Number of classes predicted by the model: 16


In [ ]:
class_labels = ['astible', 'bellflower', 'black_eyed_susan', 'calendula', 'california_poppy', 'carnation',
                'common_daisy', 'coreopsis', 'daffodil', 'dandelion', 'iris', 'magnolia', 'rose',
                'sunflower', 'tulip', 'water_lily']

In [ ]:



def predict_flower(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(288, 276))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0


    predictions = model.predict(img_array)


    predicted_class = np.argmax(predictions, axis=1)[0]


    predicted_label = class_labels[predicted_class]
    print(f"Predicted class for {os.path.basename(image_path)}: {predicted_label}")


image_path = '/content/3.jpg'

predict_flower(image_path)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Predicted class for 3.jpg: water_lily


In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d l3llff/flowers

Dataset URL: https://www.kaggle.com/datasets/l3llff/flowers
License(s): CC0-1.0
flowers.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip flowers.zip

Archive:  flowers.zip
replace flowers/astilbe/10091895024_a2ea04cda6_c.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

#Loading model
model = tf.keras.models.load_model('/content/flower_model_final.keras')

train_dir = '/content/flowers'


train_datagen = image.ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(288, 276),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

class_labels = list(train_generator.class_indices.keys())


def predict_flower(image_path):
    img = image.load_img(image_path, target_size=(288, 276))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    predictions = model.predict(img_array)


    predicted_class = np.argmax(predictions, axis=1)[0]


    predicted_label = class_labels[predicted_class]
    return predicted_label


def evaluate_accuracy(dataset_dir):
    correct_predictions = 0
    total_predictions = 0

    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)


                true_label = os.path.basename(root)


                predicted_label = predict_flower(image_path)


                if predicted_label == true_label:
                    correct_predictions += 1
                total_predictions += 1


    accuracy = correct_predictions / total_predictions
    print(f'Accuracy: {accuracy * 100:.2f}%')

evaluate_accuracy(train_dir)


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━